<a href="https://colab.research.google.com/github/vilcagamarracf/Python_Snippets/blob/main/Snippets/Snippets_geemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h2> geemap Snippets - Colaboratory 👨‍🚀</h2>
<p><br>
<img src="https://www.gislounge.com/wp-content/uploads/2018/06/google-earth-engine.jpg"height="200px" hspace="10px" vspace="0px">
<br>
<i>geemap: A Python package for interactive mapping with Google Earth Engine, ipyleaflet, and ipywidgets. <a href='https://geemap.org/'>Source</a></i>
</p>
</center>

Referencias:

- `geemap` : [Cheat Sheet](https://geemap.org/cheatsheet/) 

## Inicio rápido

1. Instalar geemap en Colab y reiniciar el kernel (sesión)

In [ ]:
%%capture
!pip install -U geemap

2. Ingresar con las credenciales de Google Earth Engine

In [ ]:
# Ingresa tus credenciales de Google Earth Engine para comenzar
import ee
ee.Authenticate()

# Inicio
ee.Initialize()

# Mensaje de login
print('\nYa estás logeado.')
print('GEE Version:', ee.__version__)

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=G1dv1MrUnjNt4Ipe4r4WPDdr_JQbvt1JiwZvJmUhZeM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjVpYn3ytnbHo_HReQDQhzM4xkYbnQBIplg_75AUr7c1AzNEVbnjQo

Successfully saved authorization token.

Ya estás logeado.
GEE Version: 0.1.283


In [ ]:
# Visualización de geemap
import geemap
Map = geemap.Map(basemap='ESRI', center=(-9.125, -74.396), zoom=5) # OpenStreetMap.Mapnik
# Map

## Explorando geemap

### Filtrado rápido

1. Dibujamos una geometría con la interfaz de geemap

In [ ]:
Map = geemap.Map(basemap='HYBRID', layer_ctrl=True) # OpenStreetMap.Mapnik
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

2. Convertimos la geometría dibujada en un `ee.geometry`. Para ello, solicitaremos la información de la geometría dibujada mediante `Map.user_rois` .

Nota:
- `Map.user_roi` guarda el último polígono dibujado mientras que 
- `Map.user_rois` guarda todos los polígonos dibujado 

In [ ]:
roi = Map.user_rois.geometry()
roi.getInfo()

{'coordinates': [[[-76.334519, -11.186794],
   [-76.334519, -10.808333],
   [-75.935007, -10.808333],
   [-75.935007, -11.186794],
   [-76.334519, -11.186794]]],
 'geodesic': False,
 'type': 'Polygon'}

3. Procedemos a filtrar un `ee.ImageCollection`

In [ ]:
#@markdown 1. Selección del producto:
Product_name = 'COPERNICUS/S2_SR'  #@param ["LANDSAT/LC08/C01/T1_SR", "COPERNICUS/S2_SR"]

#@markdown 2. Seleccionamos las fechas de interés:
Start_date = '2020-01-01'  #@param {type: "date"}
End_date   = '2020-12-31'  #@param {type: "date"}

#@markdown 3. Porcentaje de Nubosidad:
Cloud_Pixel_Percentage =     10#@param {type:"number"}

#  Parametros globales
range_date = [Start_date, End_date]

if Product_name == 'COPERNICUS/S2_SR':
  img_col = ee.ImageCollection(Product_name)\
              .filterDate(range_date[0], range_date[1])\
              .filterBounds(roi)\
              .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', Cloud_Pixel_Percentage)
else:
  img_col = ee.ImageCollection(Product_name)\
              .filterDate(range_date[0], range_date[1])\
              .filterBounds(roi)\
              .filterMetadata('CLOUD_COVER', 'less_than', Cloud_Pixel_Percentage)

print(f'# Imágenes del producto {Product_name}: {img_col.size().getInfo()}')

# Imágenes del producto COPERNICUS/S2_SR: 20


4. Visualizamos las imágenes (o imagen)

In [ ]:
# img_col.sort('CLOUDY_PIXEL_PERCENTAGE') # En caso de ordenar por porcentaje de nubosidad

# Definir simbologia RGB
ver_imagen = img_col.first().multiply(0.0001)
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 0.3,
    # 'gamma': 0.90
}

# Visualizar en el mapa
Map = geemap.Map(basemap='ESRI', layer_ctrl=True)
Map.centerObject(roi, 11) # Map.setCenter(lon, lat, zoom)
Map.addLayer(ver_imagen, vis_params, name = 'Imagen Landsat8') # Map.addLayer(image, vis_params, 'SRTM DEM', True, 0.5)
Map

Map(center=[-10.997547468014753, -76.134763], controls=(WidgetControl(options=['position', 'transparent_bg'], …

En caso de no haberse conseguido la imagen esperada, es posible modificar el área de estudio dibujada.

5. Exportar imagen

`geemap` solo permite descargas pequeñas (Total request size (3332596224 bytes) must be less than or equal to 33554432 bytes).

- https://geemap.org/notebooks/11_export_image/

In [ ]:
geemap.ee_export_image(image, filename='image.tif', region=roi, scale=30, file_per_band=False)

### Límites de Países a nivel mundial en Earth Engine

#### Filtrado
- https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filtermetadata
- https://developers.google.com/earth-engine/apidocs/ee-featurecollection-filtermetadata

`FeatureCollection.filterMetadata(name, operator, value)`

In [ ]:
# Perú como FeatureCollection
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
peru = countries.filterMetadata('country_na', 'equals', 'Peru')

In [ ]:
# Visualizar
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(peru, 5) # Map.setCenter(lon, lat, zoom)
Map.addLayer(peru, {"color" : "00FF11"}, name = 'Perú')
Map

Map(center=[-9.12511364697584, -74.39606230545007], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
peru.getInfo()['properties']

{'date_range': [1490832000000, 1490832000000],
 'description': '<p>The United States Office of the Geographer provides\nthe Large Scale International Boundary (LSIB) dataset. The detailed\nversion (2013) is derived from two other datasets: a LSIB line\nvector file and the World Vector Shorelines (WVS) from the National\nGeospatial-Intelligence Agency (NGA). The interior boundaries\nreflect U.S. government policies on boundaries, boundary disputes,\nand sovereignty. The exterior boundaries are derived from the\nWVS; however, the WVS coastline data is outdated and generally\nshifted from between several hundred meters to over a kilometer.\nEach feature is the polygonal area enclosed by interior boundaries\nand exterior coastlines where applicable, and many countries\nconsist of multiple features, one per disjoint region.</p><p>Compared with the detailed LSIB, in this simplified dataset some disjointed\nregions of each country have been reduced to a single feature.\nFurthermore, it exclud

In [ ]:
from pprint import pprint
pprint(peru.getInfo()['properties'])

{'date_range': [1490832000000, 1490832000000],
 'description': '<p>The United States Office of the Geographer provides\n'
                'the Large Scale International Boundary (LSIB) dataset. The '
                'detailed\n'
                'version (2013) is derived from two other datasets: a LSIB '
                'line\n'
                'vector file and the World Vector Shorelines (WVS) from the '
                'National\n'
                'Geospatial-Intelligence Agency (NGA). The interior '
                'boundaries\n'
                'reflect U.S. government policies on boundaries, boundary '
                'disputes,\n'
                'and sovereignty. The exterior boundaries are derived from '
                'the\n'
                'WVS; however, the WVS coastline data is outdated and '
                'generally\n'
                'shifted from between several hundred meters to over a '
                'kilometer.\n'
                'Each feature is the polygonal a

#### Exportar FeatureCollection: `geemap.ee_to_shp(PERU, filename= 'PERU.shp')`

In [ ]:
# Pasos para crear una carpeta
import os
# os.chdir('/content/')
# os.getcwd()
out_dir = os.getcwd() + '/shp_export'

# Crear la carpeta en el directorio
os.makedirs(out_dir)

In [ ]:
# Exportar shp Peru
geemap.ee_to_shp(peru, filename= out_dir+'/PERU.shp')

Generating URL ...
Please wait ...
Data downloaded to /content/shp_export/PERU.shp


In [ ]:
lista_archivos = os.listdir(out_dir)

['.ipynb_checkpoints',
 'PERU.shx',
 'PERU.fix',
 'PERU.cpg',
 'PERU.dbf',
 'PERU.prj',
 'PERU.shp']

In [ ]:
os.chdir(out_dir)

# Comprimir en un archivo zip y descargar
import zipfile

my_zip = zipfile.ZipFile('shp_export.zip', mode= 'w')

for i in lista_archivos:
  my_zip.write(i)

my_zip.close()

os.chdir('/content/')

### Conversión de código JavaScript a Python


```
# El código en javascript se escribe como string de la siguiente manera:
js_snippet = """
// Load an image.
var image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318');

// Define the visualization parameters.
var vizParams = {
  bands: ['B5', 'B4', 'B3'],
  min: 0,
  max: 0.5,
  gamma: [0.95, 1.1, 1]
};

// Center the map and display the image.
Map.setCenter(-122.1899, 37.5010, 10); // San Francisco Bay
Map.addLayer(image, vizParams, 'false color composite');
"""

# Convertir de JavaScript a Python
geemap.js_snippet_to_py(js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True)

```
**Nota**: Solo funciona en local (jupyter notebooks o jupyterlab)



También es posible descargar el código de aplicaciones de GEE y guardarlos en archivos `.js` 

In [ ]:
geemap.download_gee_app('https://google.earthengine.app/view/linked-maps', out_file=None)

The json url: https://google.earthengine.app/javascript/linked-maps-modules.json
The JavaScript is saved at: /content/linked-maps.js


In [ ]:
geemap.download_gee_app('https://minagri-geoespacial.users.earthengine.app/view/dinamicaagricolav3',
                       out_file=None)

The json url: https://minagri-geoespacial.users.earthengine.app/javascript/dinamicaagricolav3-modules.json
The JavaScript is saved at: /content/dinamicaagricolav3.js


In [ ]:
a = 5.123456789
print(round(a,4))

5.1235


### Conversión de `.shp` a datos `ee.`

**Forma 1: Con geopandas**

En la clase, el ingeniero Nino trabaja con QGIS para dibujar una zona de estudio, obtenerlo como archivo `.shp` y luego importarlo con python para poder trabajarlo como dato `ee`. (Curso: Teledetección Google Earth Engine con Python Video 6.6 Conversión de SHP a GEE)

Con geopandas se importará el archivo en formato `.shp` y se convertirá a formato geojson, para luego ser aceptado dentro de un `ee.FeatureCollection`.

In [ ]:
%%capture
!pip install geopandas

In [ ]:
import geopandas as gpd

In [ ]:
# leer archivo con geopandas.read_file()
zona_gpd = gpd.read_file('Archivo_shapefile.shp')
zona_gpd

In [ ]:
# Crear un featureCollection mediante GeoJSON: geopandas.read_file().__geo_interface__
ee_zona_gpd = ee.FeatureCollection(zona_gpd.__geo_interface__)
type(ee_zona_gpd) # ee.featurecollection.FeatureCollection

**Forma 2: Con geemap**

También es posible realizarlo rápidamente con `geemap`.

In [ ]:
path = r'asdfasd' # ruta del shapefile

ee_shp = geemap.shp_to_ee(path) # Lo convertirá en un ee.FeatureCollection
type(ee_shp) # ee.featurecollection.FeatureCollection

### Trabajar IDs de imágenes de USGS

In [ ]:
2**16

65536